In [1]:
from Bio import SeqIO
from Bio import SeqFeature
from Bio.Alphabet import IUPAC
import pandas as pd
from Levenshtein import distance
import numpy as np
import csv
from ipywidgets import IntProgress
from IPython.display import display
import multiprocessing as mp
import time
from tqdm import tqdm_notebook as tqdm
from itertools import product, combinations
from os import listdir, mkdir
import pickle
#ocf cluster

In [50]:
genomes = [f for f in listdir("data") if "_" not in f]
genomes[:7][::-1]

['Greens', 'Gorillas', 'Goldens', 'Chimps', 'Bushbaby', 'Bonobos', 'Baboons']

In [52]:
# genomes = [f for f in listdir("data") if "_" not in f]
# genomes.remove("Humans")
alu = ["AluJb", "AluJo", "AluJr", "AluJr4", "AluSc", "AluSc5", "AluSc8", "AluSg", "AluSg4", "AluSg7", "AluSp", "AluSq", "AluSq10", "AluSq2", "AluSq4", "AluSx", "AluSx1", "AluSx3", "AluSx4", "AluSz", "AluSz6", "AluY"]
# pairings = list(combinations(genomes, 2))
# print(len(pairings))
tasks = list(product(product(["Humans"], genomes[:7][::-1]), alu))
print(len(tasks))
create_tasks(tasks)

154


In [14]:
t1 = Task(tasks[1])
t2 = Task(tasks[1])
t2.update(5)
t1 == t2

True

In [54]:
tasks = [pickle.load(file=open("tasks/" + f, "rb")) for f in listdir("tasks")]
started = [task.completed for task in tasks if task.completed > 0]
finished = [task for task in tasks if task.finished()]
print(np.average(started))
df = pd.read_csv("results/"+started[0].filename()+".csv", header=None)

506.17224880382776


AttributeError: 'int' object has no attribute 'filename'

In [35]:
df.head()

,chr1,1048537,1048694,chr19,706004,706165,37,342533,TTAGCTGGGCATACTGGTGTGAGTCTGTGGTCCCAGCTACCTGCAAGGCAGAGGCAGAAGGACTGCCTGAGCCTGGGAGGTCGAGGCTGCAGTAAGCTGTGATCATGCCACTACCCTCCAGTCACGGCAACAGAGACCTTGTTTAAAAAAAAAACAAA,TTAGCTGGGCACGGTGGTGTGTGCCTGTGGTCCCAGTTACTTGGGAGGCTAAGGTGGGAGGATCGCATGAGCCTGGGAGGTTGAGGCTACAGTGAGCCGTGATCATGCCACTGCACTCCAGCCTGGGCAACAGAGCCAGAACCTGGCTCAAAATAAAATAAA
0,chr1,72351545,72351857,chr1,70320333,70320634,32,2031212,GGTCAGGTGCAGTGGGGCATGCCTATAATCCCAGCACTTTAGGAGG...,GGTCAGGTGTGGTGGGGCATGCCTATAATCCCAGCACTTTGGGAGG...
1,chr1,94371833,94372116,chr1,93377902,93378188,19,993931,GGTTGGGTACTGTGACTCACACCTAGAATCCCAGCACTTTGGGAGG...,GGCTGGTTACTGTGGCTCACACCTATAATCCCAGCACTTTGGGAGG...
2,chr1,4849609,4849794,chr1,1900940,1901125,11,2948669,AAAAAATACAAAAATTAGCTGGGCATGGTGGCATGTGCCTGTAGTC...,AAAAAAAATAAAAATTACCTGGGCATGGTGGCATGTGCCTGTAGTC...
3,chr1,19398608,19398910,chr1,17127086,17127391,37,2271522,GGCTGGGAGCAGTGGCTCATGCCTATAATCCCAGCACTGTGGGAGG...,GGCTGGGTACGGTGGCTTATGTCTGTAATCTCACTATTTTGGGAGG...
4,chr1,24248319,24248590,chr1,22038455,22038712,48,2209864,AACAGTTTGGGAGGCCAAGGTAGGCAGATCACTTGAGCCTAGGGGT...,AACAGTTTGGGAGGCCGAGGTAGGCAGGTCACTTGAGCCTAGGAGT...


In [7]:
genomes = [f.split(".")[0] for f in listdir("data") if ".fasta" in f]
alu = ["AluJb", "AluJo", "AluJr", "AluJr4", "AluSc", "AluSc5", "AluSc8", "AluSg", "AluSg4", "AluSg7", "AluSp", "AluSq", "AluSq10", "AluSq2", "AluSq4", "AluSx", "AluSx1", "AluSx3", "AluSx4", "AluSz", "AluSz6", "AluY"]
create_pickles(genomes)

In [118]:
t = Task(tasks[0])

In [119]:
pickle.dump(obj=t, file=open(t.filename(), "wb"))

In [120]:
t = pickle.load(file=open(t.filename(), "rb"))

In [18]:
#get the frequencies for each subfamily in a genome
#ex: get_frequencies("humans")
class Task():
    def __init__(self,task):
        self.species1 = task[0][0]
        self.species2 = task[0][1]
        self.subfamily = task[1]
        self.total = len(pickle.load(file=open("data/"+self.species1 + "/" + self.species1+"_"+self.subfamily + ".p", "rb")))
        self.completed = 0
    def filename(self):
        return self.species1 + "_" + self.species2 + "_" + self.subfamily + ".p"
    def update(self,amount):
        self.completed += amount
    def finished(self):
        return self.completed == self.total
    def remaining(self):
        return self.total - self.completed
    def __eq__(self, obj):
        return (self.species1 == obj.species1) & (self.species2 == obj.species2) & (self.subfamily == obj.subfamily)
def create_tasks(tasks):
#warning: only run once, will reset progress for tasks
    for task in tasks:
        t = Task(task)
        pickle.dump(t, open("tasks/"+t.filename(), "wb"))
#     task_list = [Task(task) for task in tasks]
#     pickle.dump(obj=task_list, file=open("progress.p", "wb"))
def create_pickles(genomes):
    d = {}
    for g in genomes:
        records = SeqIO.parse("data/" + g + ".fasta", "fasta")
        for a in alu:
            d[a]=[]
        for r in records:
            subfamily = r.id.split("_")[2]
            if subfamily in d.keys():
                d[subfamily].append(r)
        mkdir("data/" + g)
        for sub, seqs in d.items():
            path = "data/" + g + "/" + g + "_" + sub + ".p"
            pickle.dump(seqs, open(path, "wb"))
        
        
def get_frequencies(species):
    records = SeqIO.parse("data/" + species + ".fasta", "fasta")
    freq = {}
    for r in records:
        subfamily = r.id.split("_")[2]
        freq[subfamily] = freq.get(subfamily, 0) + 1
    #uncomment these next three lines out if you want the relative frequencies
#     factor=1.0/sum(freq.values())
#     for sub in freq:
#         freq[sub] = freq[sub]*factor
    return freq

#return a list of sequences for a given species and subfamily
#ex: get_subfamily("humans", "AluJo")
def get_subfamily(species, subfamily):
    records = SeqIO.parse("data/" + species + ".fasta", "fasta")
    return [record for record in records if subfamily in record.id]

##return a list of sequences for a given species and subfamily on a chromosome
#ex: get_subfamily_chr("humans", "AluJo", 1)
def get_subfamily_chr(species, subfamily, chromosome):
    records = get_subfamily(species, subfamily)
    return [record for record in records if "chr"+str(chromosome) in record.description]

#retrieve a sequence for a given speicies based on its description
def get_sequence(species, description):
    records = SeqIO.parse("data/" + species + ".fasta", "fasta")
    return [record for record in records if description == record.description]

#parses location information from description string
def get_location(description):
    location = description.split(' ')[1].split(':')
    start, end = location[1].split('-')
    return [location[0].split("=")[1], int(start), int(end)]

In [25]:
get_frequencies("humans")

{'Alu': 4658,
 'AluJb': 131759,
 'AluJo': 81375,
 'AluJr': 88503,
 'AluJr4': 20966,
 'AluSc': 36338,
 'AluSc5': 7018,
 'AluSc8': 23028,
 'AluSg': 38681,
 'AluSg4': 7603,
 'AluSg7': 9159,
 'AluSp': 53809,
 'AluSq': 19866,
 'AluSq10': 2165,
 'AluSq2': 63875,
 'AluSq4': 1906,
 'AluSx': 123022,
 'AluSx1': 123492,
 'AluSx3': 34198,
 'AluSx4': 11540,
 'AluSz': 107707,
 'AluSz6': 49944,
 'AluY': 110881,
 'AluYa8': 368,
 'AluYd8': 241,
 'AluYe5': 1378,
 'AluYf1': 2025,
 'AluYg6': 899,
 'AluYh9': 165,
 'AluYi6': 470,
 'AluYk11': 1341,
 'AluYk12': 219}

# Single Threaded Code

In [78]:
def generate_pairings(species1, species2, subfamily):
    species1_records = get_subfamily(species1, subfamily)
    species2_records = get_subfamily(species2, subfamily)
    filename = species1 + "_" + species1 + "_" + subfamily + ".csv"
    file=open(filename,"w+")
    wr = csv.writer(file, quoting=csv.QUOTE_ALL)
    f = IntProgress(min=0, max=len(species1_records))
    display(f)
    for sequence1 in species1_records:
        matches = [distance(str(sequence1.seq), str(sequence2.seq)) for sequence2 in species2_records]
        match = species2_records[np.argmin(matches)]
        location1 = get_location(sequence1.description)
        location2 = get_location(match.description)
        data = np.concatenate([location1, location2, [abs(location1[1] - location2[1])], [str(sequence1.seq)], [str(match.seq)]]) 
        wr.writerow(data)
        f.value += 1
    file.close()

In [ ]:
generate_pairings("humans", "chimps", "AluJo")

# Multithreaded Code

In [ ]:
# set parameters here
species1 = "humans"
species2 = "chimps"
subfamily = "AluJo"

In [ ]:
# then run this cell
print("loading " + species1 + " data...")
species1_records = get_subfamily(species1, subfamily)
print("loading " + species2 + " data...")
species2_records = get_subfamily(species2, subfamily)
generate_pairings()

In [ ]:
def find_match(sequence):
    matches = [distance(str(sequence.seq), str(sequence2.seq)) for sequence2 in species2_records]
    match = species2_records[np.argmin(matches)]
    location1 = get_location(sequence.description)
    location2 = get_location(match.description)
    data = np.concatenate([location1, location2, [abs(location1[1] - location2[1])], [str(sequence.seq)], [str(match.seq)]]) 
    return data
    
def generate_pairings():
    filename = species1 + "_" + species2 + "_" + subfamily + ".csv"
    print("starting matching")
    p = mp.Pool(mp.cpu_count() + 2)
    with open(filename, 'w+') as f:
        wr = csv.writer(f, quoting=csv.QUOTE_ALL)
        with tqdm(total=len(species1_records), unit="match") as pbar:
            for i, result in tqdm(enumerate(p.imap(func=find_match, iterable=species1_records))):
                # (filename, count) tuples from worker
                pbar.update()
                wr.writerow(result)

In [6]:
h = SeqIO.parse("data/" + "humans" + ".fasta", "fasta")

In [7]:
c = SeqIO.parse("data/" + "chimps" + ".fasta", "fasta")

In [8]:
distance(str(next(h).seq),str(next(c).seq))

63

In [79]:
a = iter([1, 2, 3, 4, 5, 6, 7, 8])

In [84]:
next(a)

2